In [2]:
import pandas as pd
from glob import glob
import plotly.graph_objects as go

# DATASET IMPORTATION

In [3]:
#IMPORTING THE TWEETS DATASET

data1 = pd.read_csv('./db/tweets_stance_sentiment_1outof4.csv',sep='~')
data2 = pd.read_csv('./db/tweets_stance_sentiment_2outof4.csv',sep='~',names=list(data1.columns.values),header=None)
data3 = pd.read_csv('./db/tweets_stance_sentiment_3outof4.csv',sep='~',names=list(data1.columns.values),header=None)
data4 = pd.read_csv('./db/tweets_stance_sentiment_4outof4.csv',sep='~',names=list(data1.columns.values),header=None)

In [4]:
print(list(data1.columns.values))

['ID', 'user_id', 't_sentiment', 't_stance']


In [5]:
#Check that all the dataset files have the same stucture

print(data1.head())
print(data2.head())
print(data3.head())
print(data4.head())

#DATA1: 682715594110689280~259896187~positive~remain      OK
#DATA2: 817805295107850240~193680125~positive~other       OK
#DATA3: 991051346517348480~1517644448~neutral~other       OK
#DATA4: 1106286989320617984~2870546121~negative~leave     OK

                   ID    user_id t_sentiment t_stance
0  682715594110689280  259896187    positive   remain
1  682715872205651968    6312362     neutral    other
2  682716086681382912  259896187    negative   remain
3  682716257402109952    6312362    positive    other
4  682716531470544896  259896187     neutral    other
                   ID             user_id t_sentiment t_stance
0  817805295107850240           193680125    positive    other
1  817790211581165440  744802422313230208    positive    leave
2  817805320227590144          1183571862    positive    other
3  817805329094311808            76005948    negative    other
4  817805334135902208           297443753    negative   remain
                   ID             user_id t_sentiment t_stance
0  991051346517348480          1517644448     neutral    other
1  991051347935006720            51854911    positive   remain
2  991051348799041536  712654044762935296     neutral    other
3  991051375969751040           141383006    n

In [ ]:
# Concatenation in a songle dataframe

frames=[data1,data2,data3,data4]
dfTweets= pd.concat(frames)

In [ ]:
#IMPORTING THE STATISTICAL DATASET

headers=['user_id','stance','sentiment','bot_score','bot_score_time','num_tweets']
dfStatistics = pd.read_csv('./db/users_stance_sentiment_botscore_tweetcounts.csv',sep='~',names=headers,header=None)

dfStatistics.head()

In [ ]:
# Check that the number of users coincides with the documention file

# number of users: 3979965 OK

# bot_score has some missing values

print(dfStatistics.count(),'\n\n',dfStatistics.dtypes)

# EXPLORATIVE ANALYSIS and DATASET REDUCTION

In [ ]:
# Let's analyze the stance distibution among the users

labels = dfStatistics.stance.value_counts().index
values = dfStatistics.stance.value_counts().values

fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.update_layout(title_text="Stance of the users")
fig.show()

print(dfStatistics.stance.value_counts())

In [ ]:
# REDUCTION1 : Drop all the rows that doesn't have bot score (missing value)

dfStatistics=dfStatistics.dropna()
print(dfStatistics.count())
print(dfStatistics.stance.value_counts())

In [ ]:
# REDUCTION 2 : Drop all the users that have stance='others'

indexes = dfStatistics[ dfStatistics['stance'] == 'others' ].index
dfStatistics.drop(indexes, inplace=True)

# Check that are only remaining users with "leave" or "remain" stance
print(dfStatistics.stance.value_counts())

In [ ]:
# REDUCTION 2 : remove the users that have written more tweets than a normal users --> 0.90 quantile truncation

leaveTweetsPerUser=dfStatistics.loc[dfStatistics['stance']=='leave'].num_tweets
remainTweetsPerUser=dfStatistics.loc[dfStatistics['stance']=='remain'].num_tweets

# mostly of the users in the DB have written only 1,2 or 3 tweets...
print(leaveTweetsPerUser.describe(),"\n")
print(remainTweetsPerUser.describe())

# Compute the 0.90 quantile
q = dfStatistics['num_tweets'].quantile(0.90)
print("\n\n\n","90% quantile = ",q,"\n\n")

# Dataset truncation
dfStatisticsFiltered=dfStatistics[dfStatistics["num_tweets"] < q]

print(dfStatisticsFiltered.describe())

In [ ]:
# Let's analyze the stance distibution among the remaining users

labels = dfStatisticsFiltered.stance.value_counts().index
values = dfStatisticsFiltered.stance.value_counts().values

fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.update_layout(title_text="Stance of the users that are not massive brexit-twitters ( count(tweets) < 6 )")
fig.show()

print(dfStatisticsFiltered.stance.value_counts())

In [ ]:
# The distribution of bot score between leavers and remainer is very similar...

leaveTweetsBotScore=dfStatisticsFiltered.loc[dfStatisticsFiltered['stance']=='leave'].bot_score
remainTweetsBotScore=dfStatisticsFiltered.loc[dfStatisticsFiltered['stance']=='remain'].bot_score

print(remainTweetsBotScore.describe(),"\n")
print(leaveTweetsBotScore.describe())


# Acccording to the fact that the 0.9 bot_score is slightly above 50%, we know that mostly of the users are not bots

q = dfStatisticsFiltered['bot_score'].quantile(0.90)
print("\n\n\n","90% quantile = ",q,"\n\n")


# REDUCTION 3: remove the users hat have the botscore above 50% --> more or less a 0.90 quantile truncation

dfStatisticsFiltered2=dfStatisticsFiltered[dfStatisticsFiltered['bot_score'] < 0.5]

print(dfStatisticsFiltered2.describe())


In [ ]:
# Let's analyze the stance distibution among the remaining users

labels = dfStatisticsFiltered2.stance.value_counts().index
values = dfStatisticsFiltered2.stance.value_counts().values

fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.update_layout(title_text="Stance of the non-robot users ( botscore < 50% )")
fig.show()

print(dfStatisticsFiltered2.stance.value_counts())

In [ ]:
# Distribution of tweets per user (bounds [1,5])

import plotly.express as px

fig = px.histogram(dfStatisticsFiltered2, x="num_tweets", color="stance")
fig.update_layout(title_text="Distribution of tweets for non-robots users ( bounds [1,5] )")
fig.show()



In [ ]:
# Now I have to filter the tweets dataset, and in particula I will only keep the tweets of the remaining users

dfTweetsFiltered=dfTweets.loc[dfTweets['user_id'].isin(dfStatisticsFiltered2['user_id'])]

In [ ]:
# Let's how many are the remaing tweets after the filtering operation

print(dfTweetsFiltered.count(),"\n\n")

print(dfTweetsFiltered.t_stance.value_counts(),"\n\n")

print(dfTweetsFiltered.t_sentiment.value_counts())

# FINAL CHECK: 817603 exacly the tweets counting in the other dataset containing the users staces --> All OK ! 

# FILTERED CSV EXPORTATIONS

In [ ]:
# Record shuffling

dfStatisticsFiltered2.sample(frac=1)

print(dfStatisticsFiltered2.stance.value_counts())

In [ ]:
# CSV exportations

dfTweetsFiltered.to_csv('./db/tweets_filtered.csv',index=False)

dfStatisticsFiltered2.to_csv('./db/users_filtered.csv',index=False)